In [1]:
# 1-1. 데이터 가져오기 

import pandas as pd 
import matplotlib.pyplot as plt 


socar = pd.read_csv('../../0. raw_data/insurance_fraud_detect_data.csv')
socar 

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


In [2]:
# 테스트 셋 분리

df = socar[socar['test_set']==0]
df_test = socar[socar['test_set']==1]

In [3]:
# test_set 분리 
X_test = df_test.drop('fraud_YN', axis = 1)
y_test = df_test['fraud_YN']

In [4]:
# train-valid 분리 

from sklearn.model_selection import train_test_split

y_target = df['fraud_YN']
X_features = df.drop('fraud_YN', axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.73794
1     0.26206
Name: fraud_YN, dtype: float64
0    99.728261
1     0.271739
Name: fraud_YN, dtype: float64


In [5]:
# 학습 / 예측 / 평가 함수 

# 평가 함수 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred):
    # confusion = confusion_matrix(y_test, pred) #오차행렬
    accuracy = accuracy_score(y_test,pred) # 정확도 
    precision = precision_score(y_test,pred) # 정밀도
    recall = recall_score(y_test,pred) # 재현율 
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred)
    return accuracy, precision, recall, f1, roc_auc

# 성능 DF 반환 

def get_result(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    return get_clf_eval(y_test, pred)

# 다수의 모델 성능 비교 

def get_result_df(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc']
    tmp = []
    for model in models:
        tmp.append(get_result(model, X_train, y_train, X_test, y_test))

    return pd.DataFrame(tmp, columns=col_names, index=model_names)


# 성능 확인 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC


# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()


# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.997283,0.0,0.0,0.0,0.500000
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.980978,0.0,0.0,0.0,0.491826


## Trial#1 
### 변수삭제	원핫인코딩	결측치처리	처리방안 	PCA	샘플링
### 삭제안함	적용안함	처리 안함	없음	2	SMOTE

In [25]:
df_1 = df.copy()

y_target = df_1['fraud_YN']
X_features = df_1.drop('fraud_YN', axis=1)

y_target = df_1['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [26]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-10.887075,-1.580856,0
12875,-10.882941,-1.683657,0
12876,-10.890038,-1.575054,0
12877,-10.883275,-1.688706,0
12878,-10.909166,0.499242,0


In [27]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996506,0.0,0.0,0.0,0.499611
SVC,0.996118,0.0,0.0,0.0,0.499416


In [28]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision    recall        f1       roc
LogiReg       0.139363   0.002700  0.857143  0.005384  0.497275
DecisionTree  0.826863   0.006742  0.428571  0.013274  0.628260
RandomFore    0.826087   0.004494  0.285714  0.008850  0.556637
LGBM          0.819488   0.004329  0.285714  0.008529  0.553328
SVC           0.139363   0.002700  0.857143  0.005384  0.497275
              Accuracy  precision    recall        f1       roc
LogiReg       0.139363   0.002700  0.857143  0.005384  0.497275
DecisionTree  0.815606   0.006329  0.428571  0.012474  0.622616
RandomFore    0.830357   0.004608  0.285714  0.009070  0.558778
LGBM          0.826475   0.002262  0.142857  0.004454  0.485598
SVC           0.139363   0.002700  0.857143  0.005384  0.497275
              Accuracy  precision    recall        f1       roc
LogiReg       0.746894   0.001546  0.142857  0.003058  0.445699
DecisionTree  0.513199   0.003978  0.714286  0.007911  0.613468
RandomFore    0.544255   0.004248  0.714

In [35]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.139363,0.002700,0.857143,0.005384,0.497275
DecisionTree,0.888975,0.010526,0.428571,0.020548,0.659401
RandomFore,0.997283,0.000000,0.000000,0.000000,0.500000
LGBM,0.995342,0.000000,0.000000,0.000000,0.499027
SVC,0.139363,0.002700,0.857143,0.005384,0.497275


## Trial#2
### 변수삭제	원핫인코딩	결측치처리	처리방안 	PCA	샘플링
### 삭제안함	적용안함	처리 안함	없음	3	SMOTE

In [36]:
df_2 = df.copy()

y_target = df_2['fraud_YN']
X_features = df_2.drop('fraud_YN', axis=1)

y_target = df_2['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [37]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,target
12874,-10.887075,-1.580856,0.932881,0
12875,-10.882941,-1.683657,-0.666691,0
12876,-10.890038,-1.575054,0.889379,0
12877,-10.883275,-1.688706,-0.661186,0
12878,-10.909166,0.499242,0.074253,0


In [38]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996118,0.0,0.0,0.0,0.499416
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [39]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision    recall        f1       roc
LogiReg       0.147127   0.002725  0.857143  0.005432  0.501168
DecisionTree  0.743012   0.004539  0.428571  0.008982  0.586220
RandomFore    0.931289   0.011494  0.285714  0.022099  0.609381
LGBM          0.920031   0.009852  0.285714  0.019048  0.603737
SVC           0.144022   0.002715  0.857143  0.005413  0.499611
              Accuracy  precision    recall        f1       roc
LogiReg       0.146351   0.002722  0.857143  0.005427  0.500779
DecisionTree  0.739130   0.004471  0.428571  0.008850  0.584274
RandomFore    0.932065   0.011628  0.285714  0.022346  0.609770
LGBM          0.930512   0.011364  0.285714  0.021858  0.608992
SVC           0.144022   0.002715  0.857143  0.005413  0.499611
              Accuracy  precision    recall        f1       roc
LogiReg       0.404115   0.003251  0.714286  0.006472  0.558778
DecisionTree  0.558230   0.004382  0.714286  0.008711  0.636045
RandomFore    0.515140   0.003994  0.714

In [45]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.140916,0.002705,0.857143,0.005393,0.498054
DecisionTree,0.736025,0.004418,0.428571,0.008746,0.582717
RandomFore,0.997283,0.000000,0.000000,0.000000,0.500000
LGBM,0.997283,0.000000,0.000000,0.000000,0.500000
SVC,0.145186,0.002719,0.857143,0.005420,0.500195


## Trial#3
### 변수삭제	원핫인코딩	결측치처리	처리방안 	PCA	샘플링
### 삭제안함	적용안함	처리 안함	없음	4	SMOTE

In [46]:
df_3 = df.copy()

y_target = df_3['fraud_YN']
X_features = df_3.drop('fraud_YN', axis=1)

y_target = df_3['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [47]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=4)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,component_4,target
12874,-10.887075,-1.580856,0.932879,-0.715386,0
12875,-10.882941,-1.683657,-0.666696,0.059066,0
12876,-10.890038,-1.575054,0.889374,-0.076420,0
12877,-10.883275,-1.688706,-0.661185,-0.079082,0
12878,-10.909166,0.499242,0.074258,-0.183386,0


In [48]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995730,0.0,0.0,0.0,0.499221
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [49]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision    recall        f1       roc
LogiReg       0.631599   0.004211  0.571429  0.008359  0.601596
DecisionTree  0.661102   0.003440  0.428571  0.006826  0.545154
RandomFore    0.962345   0.000000  0.000000  0.000000  0.482483
LGBM          0.963509   0.011236  0.142857  0.020833  0.554301
SVC           0.643245   0.004348  0.571429  0.008630  0.607435
              Accuracy  precision    recall        f1       roc
LogiReg       0.635870   0.004260  0.571429  0.008457  0.603737
DecisionTree  0.702640   0.003922  0.428571  0.007772  0.565979
RandomFore    0.960792   0.000000  0.000000  0.000000  0.481705
LGBM          0.967391   0.000000  0.000000  0.000000  0.485014
SVC           0.591227   0.003795  0.571429  0.007540  0.581355
              Accuracy  precision    recall        f1       roc
LogiReg       0.650621   0.004440  0.571429  0.008811  0.611133
DecisionTree  0.544255   0.004248  0.714286  0.008446  0.629039
RandomFore    0.564053   0.005319  0.857

In [54]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.644022,0.003275,0.428571,0.006501,0.536590
DecisionTree,0.888199,0.000000,0.000000,0.000000,0.445309
RandomFore,0.997283,0.000000,0.000000,0.000000,0.500000
LGBM,0.996894,0.000000,0.000000,0.000000,0.499805
SVC,0.650233,0.003333,0.428571,0.006615,0.539704


## Trial#4
### 변수삭제	원핫인코딩	결측치처리	처리방안 	PCA	샘플링
### 삭제안함	적용안함	처리 안함	없음	5	SMOTE

In [55]:
df_4 = df.copy()

y_target = df_4['fraud_YN']
X_features = df_4.drop('fraud_YN', axis=1)

y_target = df_4['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [56]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=5)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4','component_5']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,component_4,component_5,target
12874,-10.887075,-1.580856,0.932880,-0.715384,0.206668,0
12875,-10.882941,-1.683657,-0.666695,0.059070,-0.826409,0
12876,-10.890038,-1.575054,0.889375,-0.076416,-0.423431,0
12877,-10.883275,-1.688706,-0.661184,-0.079080,-0.856999,0
12878,-10.909166,0.499242,0.074255,-0.183391,-0.636613,0


In [57]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.995730,0.0,0.0,0.0,0.499221
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [58]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision    recall        f1       roc
LogiReg       0.684783   0.004920  0.571429  0.009756  0.628260
DecisionTree  0.547748   0.003431  0.571429  0.006820  0.559556
RandomFore    0.972438   0.000000  0.000000  0.000000  0.487544
LGBM          0.970497   0.000000  0.000000  0.000000  0.486571
SVC           0.671972   0.004728  0.571429  0.009379  0.621837
              Accuracy  precision    recall        f1       roc
LogiReg       0.685559   0.004932  0.571429  0.009780  0.628649
DecisionTree  0.696817   0.003846  0.428571  0.007624  0.563060
RandomFore    0.973214   0.000000  0.000000  0.000000  0.487933
LGBM          0.973991   0.000000  0.000000  0.000000  0.488322
SVC           0.659550   0.004556  0.571429  0.009040  0.615609
              Accuracy  precision    recall        f1       roc
LogiReg       0.803183   0.005929  0.428571  0.011696  0.616388
DecisionTree  0.559783   0.005268  0.857143  0.010471  0.708058
RandomFore    0.549301   0.005146  0.857

In [63]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.777174,0.006957,0.571429,0.013746,0.674582
DecisionTree,0.572593,0.003630,0.571429,0.007214,0.572012
RandomFore,0.997283,0.000000,0.000000,0.000000,0.500000
LGBM,0.996894,0.000000,0.000000,0.000000,0.499805
SVC,0.772904,0.006826,0.571429,0.013491,0.672441


## Trial#5
### 변수삭제	원핫인코딩	결측치처리   	처리방안 
### 삭제안함	적용안함	repair cost 	평균	

############## 보간법을 사용하여 null 값 처리 

In [ ]:
df_5 = df.copy()

y_target = df_5['fraud_YN']
X_features = df_5.drop('fraud_YN', axis=1)

y_target = df_5['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [107]:
from sklearn.impute import KNNImputer

X_features['repair_cost'].replace(0, np.nan, inplace=True)
imputer = KNNImputer(n_neighbors=2)

#repair_cost에만 결측치 처리해줌 
X_features_filled = pd.DataFrame(imputer.fit_transform(X_features))
X_features_filled


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2.00000,1.00000,3.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
1,3.00000,1.00000,3.00000,0.00000,4.00000,0.00000,0.00000,2.00000,0.00000,0.00000,...,0.00000,3.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
2,1.00000,0.00000,1.00000,0.00000,1.00000,0.00000,100.00000,1.00000,0.00000,0.00000,...,1209410.00000,1.00000,0.00000,0.00000,2.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
3,1.00000,1.00000,3.00000,0.00000,4.00000,0.00000,100.00000,3.00000,0.00000,0.00000,...,1217280.00000,0.00000,0.00000,0.00000,1.00000,1.00000,2.00000,1.00000,2.00000,0.00000
4,2.00000,0.00000,4.00000,0.00000,4.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.00000,1.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,2.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12874,2.00000,0.00000,3.00000,1.00000,4.00000,0.00000,100.00000,1.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12875,2.00000,0.00000,2.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,1.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12876,2.00000,0.00000,2.00000,1.00000,4.00000,0.00000,100.00000,1.00000,1.00000,0.00000,...,0.00000,2.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12877,2.00000,0.00000,2.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,4.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000


In [108]:
X_features

,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,socarsave,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,2,1,3,0,2,0,100,2,0,0,...,0,1,0,0,1,0,0,0,-1,0
3,3,1,3,0,4,0,0,2,0,0,...,0,3,1,0,1,0,0,0,-1,0
4,1,0,1,0,1,0,100,1,0,0,...,1209410,1,0,0,2,0,0,0,-1,0
6,1,1,3,0,4,0,100,3,0,0,...,1217280,0,0,0,1,1,2,1,2,0
8,2,0,4,0,4,0,0,1,0,0,...,0,1,0,0,1,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15994,2,0,3,1,4,0,100,1,0,0,...,0,0,0,0,1,0,0,0,-1,0
15995,2,0,2,0,2,0,100,2,0,0,...,0,1,1,0,1,0,0,0,-1,0
15996,2,0,2,1,4,0,100,1,1,0,...,0,2,1,0,1,0,0,0,-1,0
15998,2,0,2,0,2,0,100,2,0,0,...,0,4,1,0,1,0,0,0,-1,0


In [109]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features_filled)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-10.88514,-1.52303,0
12875,-10.88575,-1.68244,0
12876,-10.88796,-1.53311,0
12877,-10.88830,-1.63437,0
12878,-10.90643,0.45217,0


In [110]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99651,0.00000,0.00000,0.00000,0.49961
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99573,0.00000,0.00000,0.00000,0.49922
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [111]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.13936    0.00270 0.85714 0.00538 0.49728
DecisionTree   0.55901    0.00352 0.57143 0.00699 0.56520
RandomFore     0.83657    0.00478 0.28571 0.00941 0.56189
LGBM           0.83113    0.00463 0.28571 0.00911 0.55917
SVC            0.13936    0.00270 0.85714 0.00538 0.49728
              Accuracy  precision  recall      f1     roc
LogiReg        0.13936    0.00270 0.85714 0.00538 0.49728
DecisionTree   0.55978    0.00352 0.57143 0.00701 0.56559
RandomFore     0.83851    0.00484 0.28571 0.00952 0.56286
LGBM           0.82570    0.00448 0.28571 0.00883 0.55644
SVC            0.13936    0.00270 0.85714 0.00538 0.49728
              Accuracy  precision  recall      f1     roc
LogiReg        0.74961    0.00156 0.14286 0.00309 0.44706
DecisionTree   0.52135    0.00324 0.57143 0.00645 0.54632
RandomFore     0.51592    0.00321 0.57143 0.00637 0.54360
LGBM           0.39325    0.00319 0.71429 0.00636 0.55333
SVC           

In [116]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.13936,0.00270,0.85714,0.00538,0.49728
DecisionTree,0.89713,0.00763,0.28571,0.01487,0.59225
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.00000,0.00000,0.00000,0.49981
SVC,0.13936,0.00270,0.85714,0.00538,0.49728


## Trial#6 
### 변수삭제	원핫인코딩	결측치처리   	처리방안 PCA 
### 삭제안함	적용안함	repair cost 	KNN 	3 

In [118]:
df_6 = df.copy()

y_target = df_6['fraud_YN']
X_features = df_6.drop('fraud_YN', axis=1)

y_target = df_6['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [119]:
from sklearn.impute import KNNImputer

X_features['repair_cost'].replace(0, np.nan, inplace=True)
imputer = KNNImputer(n_neighbors=2)

#repair_cost에만 결측치 처리해줌 
X_features_filled = pd.DataFrame(imputer.fit_transform(X_features))
X_features_filled


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2.00000,1.00000,3.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
1,3.00000,1.00000,3.00000,0.00000,4.00000,0.00000,0.00000,2.00000,0.00000,0.00000,...,0.00000,3.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
2,1.00000,0.00000,1.00000,0.00000,1.00000,0.00000,100.00000,1.00000,0.00000,0.00000,...,1209410.00000,1.00000,0.00000,0.00000,2.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
3,1.00000,1.00000,3.00000,0.00000,4.00000,0.00000,100.00000,3.00000,0.00000,0.00000,...,1217280.00000,0.00000,0.00000,0.00000,1.00000,1.00000,2.00000,1.00000,2.00000,0.00000
4,2.00000,0.00000,4.00000,0.00000,4.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.00000,1.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,2.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12874,2.00000,0.00000,3.00000,1.00000,4.00000,0.00000,100.00000,1.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12875,2.00000,0.00000,2.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,1.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12876,2.00000,0.00000,2.00000,1.00000,4.00000,0.00000,100.00000,1.00000,1.00000,0.00000,...,0.00000,2.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12877,2.00000,0.00000,2.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,4.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000


In [120]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features_filled)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,target
12874,-10.88514,-1.52303,0.97035,0
12875,-10.88575,-1.68244,-0.27594,0
12876,-10.88796,-1.53311,0.81870,0
12877,-10.88830,-1.63437,0.10646,0
12878,-10.90643,0.45217,-0.34569,0


In [121]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99728,0.00000,0.00000,0.00000,0.50000
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.00000,0.00000,0.00000,0.49981
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [122]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.13936    0.00270 0.85714 0.00538 0.49728
DecisionTree   0.85443    0.00270 0.14286 0.00531 0.49961
RandomFore     0.93323    0.00599 0.14286 0.01149 0.53912
LGBM           0.91925    0.00493 0.14286 0.00952 0.53211
SVC            0.13936    0.00270 0.85714 0.00538 0.49728
              Accuracy  precision  recall      f1     roc
LogiReg        0.13936    0.00270 0.85714 0.00538 0.49728
DecisionTree   0.78494    0.00363 0.28571 0.00717 0.53601
RandomFore     0.93168    0.00585 0.14286 0.01124 0.53834
LGBM           0.91615    0.00474 0.14286 0.00917 0.53056
SVC            0.13936    0.00270 0.85714 0.00538 0.49728
              Accuracy  precision  recall      f1     roc
LogiReg        0.50194    0.00312 0.57143 0.00620 0.53659
DecisionTree   0.53533    0.00334 0.57143 0.00664 0.55333
RandomFore     0.42197    0.00402 0.85714 0.00799 0.63896
LGBM           0.45691    0.00215 0.42857 0.00427 0.44278
SVC           

In [127]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.33773,0.00293,0.71429,0.00583,0.52550
DecisionTree,0.87034,0.00304,0.14286,0.00595,0.50759
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99651,0.00000,0.00000,0.00000,0.49961
SVC,0.13936,0.00270,0.85714,0.00538,0.49728


## Trial#6
## 변수삭제 원핫인코딩 결측치처리 처리방안 PCA
## 삭제안함 적용안함 repair cost KNN 4

In [128]:
df_7 = df.copy()

y_target = df_7['fraud_YN']
X_features = df_7.drop('fraud_YN', axis=1)

y_target = df_7['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [129]:
from sklearn.impute import KNNImputer

X_features['repair_cost'].replace(0, np.nan, inplace=True)
imputer = KNNImputer(n_neighbors=2)

#repair_cost에만 결측치 처리해줌 
X_features_filled = pd.DataFrame(imputer.fit_transform(X_features))
X_features_filled


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2.00000,1.00000,3.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
1,3.00000,1.00000,3.00000,0.00000,4.00000,0.00000,0.00000,2.00000,0.00000,0.00000,...,0.00000,3.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
2,1.00000,0.00000,1.00000,0.00000,1.00000,0.00000,100.00000,1.00000,0.00000,0.00000,...,1209410.00000,1.00000,0.00000,0.00000,2.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
3,1.00000,1.00000,3.00000,0.00000,4.00000,0.00000,100.00000,3.00000,0.00000,0.00000,...,1217280.00000,0.00000,0.00000,0.00000,1.00000,1.00000,2.00000,1.00000,2.00000,0.00000
4,2.00000,0.00000,4.00000,0.00000,4.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.00000,1.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,2.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12874,2.00000,0.00000,3.00000,1.00000,4.00000,0.00000,100.00000,1.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12875,2.00000,0.00000,2.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,1.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12876,2.00000,0.00000,2.00000,1.00000,4.00000,0.00000,100.00000,1.00000,1.00000,0.00000,...,0.00000,2.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12877,2.00000,0.00000,2.00000,0.00000,2.00000,0.00000,100.00000,2.00000,0.00000,0.00000,...,0.00000,4.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000


In [131]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features_filled)

pca = PCA(n_components=4)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,component_4,target
12874,-10.88514,-1.52303,0.97034,-0.72588,0
12875,-10.88575,-1.68244,-0.27594,0.05003,0
12876,-10.88796,-1.53311,0.81870,-0.10719,0
12877,-10.88830,-1.63437,0.10646,-0.08724,0
12878,-10.90643,0.45217,-0.34569,-0.20355,0


In [132]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99651,0.00000,0.00000,0.00000,0.49961
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99728,0.00000,0.00000,0.00000,0.50000
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [133]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.64092    0.00432 0.57143 0.00857 0.60627
DecisionTree   0.47516    0.00369 0.71429 0.00734 0.59439
RandomFore     0.96196    0.00000 0.00000 0.00000 0.48229
LGBM           0.96584    0.00000 0.00000 0.00000 0.48424
SVC            0.62500    0.00414 0.57143 0.00821 0.59829
              Accuracy  precision  recall      f1     roc
LogiReg        0.64286    0.00434 0.57143 0.00862 0.60724
DecisionTree   0.59977    0.00291 0.42857 0.00579 0.51440
RandomFore     0.96118    0.00000 0.00000 0.00000 0.48190
LGBM           0.96623    0.00000 0.00000 0.00000 0.48443
SVC            0.63781    0.00428 0.57143 0.00850 0.60471
              Accuracy  precision  recall      f1     roc
LogiReg        0.65528    0.00450 0.57143 0.00893 0.61347
DecisionTree   0.68866    0.00498 0.57143 0.00988 0.63021
RandomFore     0.51281    0.00318 0.57143 0.00633 0.54204
LGBM           0.51398    0.00398 0.71429 0.00792 0.61386
SVC           

In [138]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.63859,0.00323,0.42857,0.00640,0.53387
DecisionTree,0.88005,0.00974,0.42857,0.01905,0.65492
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.00000,0.00000,0.00000,0.49981
SVC,0.65256,0.00336,0.42857,0.00666,0.54087


In [ ]:
## 다시 

In [145]:
df_8 = df.copy()

df_8 = df_8.drop(columns=['sharing_type', 'has_previous_accident','b2b','accident_ratio','socarpass', 'car_part2'])
df_8

y_target = df_8['fraud_YN']
X_features = df_8.drop('fraud_YN', axis=1)

y_target = df_8['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [146]:
from sklearn.impute import KNNImputer

X_features['repair_cost'].replace(0, np.nan, inplace=True)
imputer = KNNImputer(n_neighbors=2)

#repair_cost에만 결측치 처리해줌 
X_features_filled = pd.DataFrame(imputer.fit_transform(X_features))
X_features_filled


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2.00000,3.00000,2.00000,2.00000,0.00000,4.00000,2.00000,4.00000,130254.67800,0.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
1,3.00000,3.00000,4.00000,2.00000,0.00000,3.00000,3.00000,5.00000,1341009.16300,0.00000,3.00000,1.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
2,1.00000,1.00000,1.00000,1.00000,0.00000,1.00000,1.00000,1.00000,524030.56960,1209410.00000,1.00000,0.00000,2.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
3,1.00000,3.00000,4.00000,3.00000,0.00000,6.00000,3.00000,1.00000,202768.67835,1217280.00000,0.00000,0.00000,1.00000,1.00000,2.00000,1.00000,2.00000,0.00000
4,2.00000,4.00000,4.00000,1.00000,0.00000,4.00000,2.00000,6.00000,502473.29500,0.00000,1.00000,0.00000,1.00000,1.00000,1.00000,1.00000,2.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12874,2.00000,3.00000,4.00000,1.00000,0.00000,4.00000,1.00000,4.00000,667293.08700,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12875,2.00000,2.00000,2.00000,2.00000,0.00000,1.00000,1.00000,1.00000,316638.97035,0.00000,1.00000,1.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12876,2.00000,2.00000,4.00000,1.00000,0.00000,5.00000,2.00000,6.00000,618439.44180,0.00000,2.00000,1.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000
12877,2.00000,2.00000,2.00000,2.00000,0.00000,6.00000,1.00000,2.00000,421042.54785,0.00000,4.00000,1.00000,1.00000,0.00000,0.00000,0.00000,-1.00000,0.00000


In [147]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features_filled)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,component_3,target
12874,-0.98565,-0.46745,-0.05122,0
12875,-1.08891,1.15669,-1.19569,0
12876,-1.02844,-0.17818,1.22482,0
12877,-1.24712,0.33719,0.43588,0
12878,-1.17391,-1.50330,1.68883,0


In [148]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)

results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99728,0.00000,0.00000,0.00000,0.50000
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.00000,0.00000,0.00000,0.49981
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [149]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.71157,0.00538,0.57143,0.01065,0.64169
DecisionTree,0.41071,0.00263,0.57143,0.00524,0.49085
RandomFore,0.90839,0.00433,0.14286,0.00840,0.52666
LGBM,0.90994,0.00000,0.00000,0.00000,0.45621
SVC,0.71778,0.00549,0.57143,0.01088,0.64480


## Trial#9
## 변수삭제 원핫인코딩 결측치처리 처리방안 PCA
## 동일분포 적용안함   처리안함           3


In [151]:
df_9 = df.copy()

y_target = df_9['fraud_YN']
X_features = df_9.drop(columns= ['fraud_YN','sharing_type','has_previous_accident','socarpass','car_part2'], axis=1)

y_target = df_9['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [172]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2',]

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,target
0,-10.87773,-1.68125,-0.44408,0
1,89.08887,-0.44721,3.48782,0
2,-10.91937,0.59531,0.76393,0
3,-10.86117,0.62668,-1.25494,1
4,89.14690,-0.59167,-0.51124,0


In [173]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99612,0.00000,0.00000,0.00000,0.49942
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.33333,0.14286,0.20000,0.57104
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [174]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.14713    0.00272 0.85714 0.00543 0.50117
DecisionTree   0.75233    0.00315 0.28571 0.00623 0.51966
RandomFore     0.93168    0.00585 0.14286 0.01124 0.53834
LGBM           0.92352    0.01031 0.28571 0.01990 0.60549
SVC            0.14441    0.00272 0.85714 0.00542 0.49981
              Accuracy  precision  recall      f1     roc
LogiReg        0.14635    0.00272 0.85714 0.00543 0.50078
DecisionTree   0.74806    0.00310 0.28571 0.00613 0.51752
RandomFore     0.93595    0.00625 0.14286 0.01198 0.54048
LGBM           0.93750    0.01266 0.28571 0.02424 0.61250
SVC            0.14480    0.00272 0.85714 0.00542 0.50000
              Accuracy  precision  recall      f1     roc
LogiReg        0.40373    0.00325 0.71429 0.00647 0.55858
DecisionTree   0.50970    0.00473 0.85714 0.00941 0.68295
RandomFore     0.50000    0.00464 0.85714 0.00923 0.67808
LGBM           0.44953    0.00352 0.71429 0.00700 0.58155
SVC           

## Trial#9
## 변수삭제 원핫인코딩 결측치처리 처리방안 PCA
## 동일분포 적용안함   처리안함           3


In [160]:
df_10 = df.copy()

y_target = df_10['fraud_YN']
X_features = df_10.drop(columns= ['fraud_YN','sharing_type','has_previous_accident','socarpass','car_part2'], axis=1)

y_target = df_10['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [162]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,target
0,-10.87773,-1.68125,-0.44408,0
1,89.08887,-0.44721,3.48782,0
2,-10.91937,0.59531,0.76393,0
3,-10.86117,0.62668,-1.25494,1
4,89.14690,-0.59167,-0.51124,0


In [163]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99612,0.00000,0.00000,0.00000,0.49942
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.33333,0.14286,0.20000,0.57104
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [164]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.14713    0.00272 0.85714 0.00543 0.50117
DecisionTree   0.75233    0.00315 0.28571 0.00623 0.51966
RandomFore     0.93129    0.00581 0.14286 0.01117 0.53815
LGBM           0.92974    0.01124 0.28571 0.02162 0.60860
SVC            0.14441    0.00272 0.85714 0.00542 0.49981
              Accuracy  precision  recall      f1     roc
LogiReg        0.14635    0.00272 0.85714 0.00543 0.50078
DecisionTree   0.74806    0.00310 0.28571 0.00613 0.51752
RandomFore     0.93672    0.00633 0.14286 0.01212 0.54087
LGBM           0.93828    0.01282 0.28571 0.02454 0.61288
SVC            0.14441    0.00272 0.85714 0.00542 0.49981
              Accuracy  precision  recall      f1     roc
LogiReg        0.40373    0.00325 0.71429 0.00647 0.55858
DecisionTree   0.50970    0.00473 0.85714 0.00941 0.68295
RandomFore     0.50000    0.00464 0.85714 0.00923 0.67808
LGBM           0.44953    0.00352 0.71429 0.00700 0.58155
SVC           

In [169]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.14092,0.00271,0.85714,0.00539,0.49805
DecisionTree,0.70419,0.00264,0.28571,0.00522,0.49552
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.00000,0.00000,0.00000,0.49981
SVC,0.13975,0.00270,0.85714,0.00539,0.49747


## Trial#10
## 변수삭제 원핫인코딩 결측치처리 처리방안 PCA
## 동일분포 적용안함   처리안함           4


In [176]:
df_10 = df.copy()

y_target = df_10['fraud_YN']
X_features = df_10.drop(columns= ['fraud_YN','sharing_type','has_previous_accident','socarpass','car_part2'], axis=1)

y_target = df_10['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [177]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=4)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,component_4,target
0,-10.87773,-1.68125,-0.44408,-0.78743,0
1,89.08887,-0.44721,3.48782,-1.10475,0
2,-10.91937,0.59531,0.76394,0.81186,0
3,-10.86117,0.62668,-1.25494,1.74966,1
4,89.14690,-0.59167,-0.51124,0.51621,0


In [178]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99573,0.00000,0.00000,0.00000,0.49922
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99728,0.00000,0.00000,0.00000,0.50000
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [179]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.64441    0.00436 0.57143 0.00866 0.60802
DecisionTree   0.66498    0.00233 0.28571 0.00461 0.47587
RandomFore     0.96234    0.00000 0.00000 0.00000 0.48248
LGBM           0.96157    0.00000 0.00000 0.00000 0.48209
SVC            0.59550    0.00384 0.57143 0.00762 0.58350
              Accuracy  precision  recall      f1     roc
LogiReg        0.64713    0.00440 0.57143 0.00872 0.60938
DecisionTree   0.70846    0.00267 0.28571 0.00530 0.49766
RandomFore     0.96118    0.00000 0.00000 0.00000 0.48190
LGBM           0.96545    0.00000 0.00000 0.00000 0.48404
SVC            0.68284    0.00489 0.57143 0.00970 0.62729
              Accuracy  precision  recall      f1     roc
LogiReg        0.65450    0.00449 0.57143 0.00891 0.61308
DecisionTree   0.56871    0.00538 0.85714 0.01069 0.71253
RandomFore     0.55008    0.00515 0.85714 0.01025 0.70319
LGBM           0.57958    0.00369 0.57143 0.00733 0.57552
SVC           

In [184]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.64674,0.00439,0.57143,0.00871,0.60919
DecisionTree,0.88820,0.00000,0.00000,0.00000,0.44531
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99689,0.00000,0.00000,0.00000,0.49981
SVC,0.63509,0.00425,0.57143,0.00844,0.60335


In [181]:
results_ads

,Accuracy,precision,recall,f1,roc
LogiReg,0.64713,0.00440,0.57143,0.00872,0.60938
DecisionTree,0.70846,0.00267,0.28571,0.00530,0.49766
RandomFore,0.96118,0.00000,0.00000,0.00000,0.48190
LGBM,0.96545,0.00000,0.00000,0.00000,0.48404
SVC,0.68284,0.00489,0.57143,0.00970,0.62729


## Trial#11
## 변수삭제 원핫인코딩 결측치처리 처리방안 PCA
## 동일분포 적용안함   처리안함           5


In [185]:
df_11 = df.copy()

y_target = df_11['fraud_YN']
X_features = df_11.drop(columns= ['fraud_YN','sharing_type','has_previous_accident','socarpass','car_part2'], axis=1)

y_target = df_11['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [186]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=5)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3','component_4','component_5']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.head()


,component_1,component_2,component_3,component_4,component_5,target
0,-10.87773,-1.68125,-0.44408,-0.78743,0.02786,0
1,89.08887,-0.44721,3.48782,-1.10475,-0.44019,0
2,-10.91937,0.59531,0.76394,0.81186,-1.57318,0
3,-10.86117,0.62668,-1.25494,1.74966,2.71913,1
4,89.14690,-0.59167,-0.51124,0.51621,2.26755,0


In [187]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.99728,0.00000,0.00000,0.00000,0.50000
DecisionTree,0.99573,0.00000,0.00000,0.00000,0.49922
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99728,0.00000,0.00000,0.00000,0.50000
SVC,0.99728,0.00000,0.00000,0.00000,0.50000


In [188]:
#### 샘플링별 확인 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler

smt = SMOTE(random_state=13)
X_train_smt, y_train_smt = smt.fit_sample(X_train, y_train)

ads = ADASYN(random_state=13)
X_train_ads, y_train_ads = ads.fit_sample(X_train, y_train)

rud = RandomUnderSampler(random_state=13)
X_train_rud, y_train_rud = rud.fit_sample(X_train, y_train)

sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)

rov = RandomOverSampler(random_state=13)
X_train_rov, y_train_rov = rov.fit_sample(X_train, y_train)

model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']

results_smt = get_result_df(models, model_names, X_train_smt, y_train_smt, X_valid, y_valid)
results_ads = get_result_df(models, model_names, X_train_ads, y_train_ads, X_valid, y_valid)
results_rud = get_result_df(models, model_names, X_train_rud, y_train_rud, X_valid, y_valid)
results_sme = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)
results_rov = get_result_df(models, model_names, X_train_rov, y_train_rov, X_valid, y_valid)


print(results_smt)
print(results_ads)
print(results_rud)
print(results_sme)
print(results_rov)

              Accuracy  precision  recall      f1     roc
LogiReg        0.68556    0.00493 0.57143 0.00978 0.62865
DecisionTree   0.60443    0.00295 0.42857 0.00585 0.51674
RandomFore     0.97438    0.00000 0.00000 0.00000 0.48852
LGBM           0.97477    0.00000 0.00000 0.00000 0.48871
SVC            0.69410    0.00507 0.57143 0.01005 0.63293
              Accuracy  precision  recall      f1     roc
LogiReg        0.68789    0.00497 0.57143 0.00985 0.62982
DecisionTree   0.66848    0.00352 0.42857 0.00698 0.54885
RandomFore     0.97554    0.00000 0.00000 0.00000 0.48910
LGBM           0.97710    0.00000 0.00000 0.00000 0.48988
SVC            0.67585    0.00478 0.57143 0.00949 0.62378
              Accuracy  precision  recall      f1     roc
LogiReg        0.80318    0.00593 0.42857 0.01170 0.61639
DecisionTree   0.56871    0.00538 0.85714 0.01069 0.71253
RandomFore     0.55901    0.00526 0.85714 0.01045 0.70767
LGBM           0.57182    0.00452 0.71429 0.00898 0.64286
SVC           

In [193]:
results_rov

,Accuracy,precision,recall,f1,roc
LogiReg,0.77989,0.00704,0.57143,0.01391,0.67594
DecisionTree,0.57182,0.00272,0.42857,0.00541,0.50039
RandomFore,0.99728,0.00000,0.00000,0.00000,0.50000
LGBM,0.99728,0.00000,0.00000,0.00000,0.50000
SVC,0.79387,0.00566,0.42857,0.01117,0.61172
